# ArchiCrop tutorial - Maize

This notebook is a tutorial for executing ArchiCrop model for maize.

## 0. Imports

In [4]:
import numpy as np
from oawidgets.plantgl import *
from openalea.plantgl.all import Material, Color3, Scene

from openalea.archicrop.stics_io import read_sti_file, read_xml_file
from openalea.archicrop.archicrop import ArchiCrop
from openalea.archicrop.display import build_scene, display_scene
%gui qt

## 1. Retrieve crop model parameters about management

Retrieve crop model parameters regarding the spatial (and temporal) configuration of the crop.

In [7]:
file_tec_xml = '../data/Mais_tec.xml'
params_tec = ['densitesem', 'interrang']
tec_stics = read_xml_file(file_tec_xml, params_tec)
sowing_density = tec_stics['densitesem']
inter_row = 0.4

## 2. Retrieve crop model growth and senescence dynamics

From crop-scale data either measured or simulated with a crop model, generate a dictionnary of dictionnaries for each day with the following values :
 - "Thermal time" (float): cumulated thermal time from beginning of simulation to current day (in °C.day)
 - "Phenology" (str): current phenological stage (germination, juvenile, exponential or repro)
 - "Plant leaf area" (float): plant leaf area (in cm²) 
 - "Leaf area increment" (float): leaf area increment compared to previous day (in cm²) 
 - "Plant senescent leaf area" (float): senescent plant leaf area (in cm²) 
 - "Senescent leaf area increment" (float): senescent leaf area increment compared to previous day (in cm²) 
 - "Plant height" (float): plant height (in cm) 
 - "Height increment" (float): height increment compared to previous day (in cm).

In [8]:
stics_output_file = '../data/mod_smaize.sti'
daily_dynamics = read_sti_file(stics_output_file, sowing_density)

# Time series 
# for thermal time, plant leaf area, plant senescent leaf area and plant height
thermal_time = [value["Thermal time"] for value in daily_dynamics.values()]
leaf_area_plant = [value["Plant leaf area"] for value in daily_dynamics.values()]
sen_leaf_area_plant = [value["Plant senescent leaf area"] for value in daily_dynamics.values()]
height_canopy = [value["Plant height"] for value in daily_dynamics.values()]

In [9]:
file_plt_xml = '../data/corn_plt.xml'
params_sen = ['durvieF', 'ratiodurvieI']
sen_stics = read_xml_file(file_plt_xml, params_sen)
lifespan = sen_stics['durvieF'] # leaf lifespan from appearance in exponential phase
lifespan_early = sen_stics['ratiodurvieI'] * lifespan # leaf lifespan from appearance in juvenile phase

## 3. Set plant architectural parameters

Set topological, geometrical and developmental parameters, in a range corresponding a given species, found in literature.

In [10]:
archi = dict(
    nb_phy=18, # number of phytomers on the main stem # [8,13,18]
    nb_short_phy=4,
    
    # Stem
    height=3*max(height_canopy), # potential plant height
    stem_q=1.0, # parameter for ligule height geometric distribution along axis
    diam_base=2.5, # stem base diameter
    diam_top=1.5, # stem top diameter

    # Leaf area distribution along the stem  
    leaf_area=1.2*max(leaf_area_plant), # potential plant leaf area
    rmax=0.55, # relative position of largest leaf on the stem # [0.72, 0.8, 0.55]
    skew=0.005, # skewness for leaf area distribution along axis

    # blade area
    wl=0.12, # leaf blade width-to-length ratio 
    klig=0.6, # parameter for leaf blade shape
    swmax=0.55, # relative position of maximal blade width
    f1=0.64, # parameter for leaf blade shape
    f2=0.92, # parameter for leaf blade shape

    # blade curvature
    insertion_angle=35, # leaf blade insertion angle
    scurv=0.7, #  relative position of inflexion point
    curvature=120, # leaf blade insertion-to-tip angle
    phyllotactic_angle=180, # phyllotactic angle
    phyllotactic_deviation=10, # half-deviation to phyllotactic angle

    # Development
    phyllochron=30, # phyllochron, i.e. stem element appearance rate
    plastochron=40, # plastochron, i.e. leaf blade appearance rate

    # Senescence 
    leaf_lifespan = [lifespan_early, lifespan], # leaf lifespan from appearance

    # Tillering
    nb_tillers=0, # number of tillers
    tiller_delay=1, # delay, as factor of phyllochron, between the appearance of a phytomer and the appearance of its tiller
    tiller_angle=30,
    reduction_factor=1, # reduction factor between tillers of consecutive order

    plant_orientation=90 #20
)

## 4. Instantiate ArchiCrop object

In [11]:
maize = ArchiCrop(daily_dynamics=daily_dynamics, **archi)

## 5. Generate a potential plant 

Generate a potential plant from architectural parameters. 

This creates an MTG (Multi-scale Tree Graph, cf [doc](https://mtg.readthedocs.io/en/latest/user/tutorial.html)) as a property of the object plant.

In [12]:
maize.generate_potential_plant()

## 6. Simulate growth and senescence of the plant

Simulate growth and senescence of this plant following the daily dynamics.

In [13]:
growing_maize = maize.grow_plant()

## 7. Plot 3D scene

In [16]:
times = [t for i,t in enumerate(thermal_time) if i%10 == 0]
mean_time = sum(times) / len(times)
positions = [ (0, 1*(t-mean_time), 0) for t in times]
nice_green = Color3((50, 100, 0))
scene, _ = build_scene([g for i,g in enumerate(list(growing_maize.values())) if i%10 == 0], position=positions, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))
PlantGL(scene)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [14]:
a = {'nb_short_phy': 3, 'short_phy_height': 3, 'diam_base': 2.2, 'diam_top': 1.2, 'klig': 0.6, 'swmax': 0.55, 'f1': 0.64, 'f2': 0.92, 'insertion_angle': 35, 'scurv': 0.7, 'curvature': 120, 'phyllotactic_angle': 137.5, 'phyllotactic_deviation': 0, 'nb_tillers': 0, 'tiller_delay': 1, 'tiller_angle': 30, 'reduction_factor': 1, 'plant_orientation': 0, 'leaf_lifespan': 200.0, 'nb_phy': 22, 'stem_q': 1.0184, 'rmax': 0.7412, 'skew': 0.0076, 'wl': 0.1117, 'phyllochron': 46.1295, 'plastochron': 57.67, 'height': 250.4644, 'leaf_area': 6443.8613}
plant = ArchiCrop(**a, daily_dynamics=daily_dynamics)
plant.generate_potential_plant()
g = plant.g
growing_plant = plant.grow_plant()
# scene, _ = build_scene(g)
# PlantGL(scene)

In [69]:
# times = [i for i,t in enumerate(thermal_time) if i%12==0]
times = [268.2979, 351.0863, 496.6405, 697.7067, 1041.6067, 1444.2067]
mean_time = sum(times) / len(times)
position = [ (0, 0.7*(t-(times[-1]-times[0])), 0) for t in times]
scene, _ = build_scene([g for i,g in growing_maize.items() if i in times], position=position, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))

In [70]:
Viewer.display(scene)